# Lab | Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')

In [3]:
#!pip install pandas

In [4]:
import pandas as pd

# Specify the URL of the CSV file
url = 'https://github.com/ironhack-labs/lab-chains-in-langchain/raw/main/data/Data.csv'

# Read CSV with specific parameters
try:
    df = pd.read_csv(url, delimiter=',')  # Adjust delimiter if necessary
    print("CSV file loaded successfully.")
    print(df.head())  # Print first few rows to verify
except Exception as e:
    print(f"Error loading CSV file: {e}")


CSV file loaded successfully.
                   Product                                             Review
0     Queen Size Sheet Set  I ordered a king size set. My only criticism w...
1   Waterproof Phone Pouch  I loved the waterproof sac, although the openi...
2      Luxury Air Mattress  This mattress had a small hole in the top of i...
3           Pillows Insert  This is the best throw pillow fillers on Amazo...
4  Milk Frother Handheld\n   I loved this product. But they only seem to l...


In [5]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


## LLMChain

In [6]:
#!pip install langchain-openai langchain-core langchain

In [7]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [8]:
#Replace None by your own value and justify
llm = ChatOpenAI(temperature=0.5)

# Justification:
# A temperature of 0.5 balances between generating diverse responses
# and maintaining coherence. It introduces a moderate level of randomness,
# allowing for varied responses without straying too far from relevant content.

In [9]:
# Define a variable placeholder for product description
product_description = "{product}"

# Construct the prompt template
prompt = ChatPromptTemplate.from_template(
    f"Write a query that would take a variable to describe any product.\n\n"
    f"Describe the product: {product_description}\n\n"
    f"Example: Describe the features, specifications, and benefits of {product_description}."
)
print("Prompt template constructed successfully.")


Prompt template constructed successfully.


In [10]:
chain = LLMChain(llm=llm, prompt=prompt)

/opt/anaconda3/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [11]:
product = "Queen Size Sheet Set"
#chain.run(product)
llm.invoke(product)


AIMessage(content='A queen size sheet set typically includes a fitted sheet, a flat sheet, and two pillowcases. These sheet sets are designed to fit a standard queen size mattress, which measures 60 inches by 80 inches. Queen size sheet sets come in a variety of materials, such as cotton, microfiber, and linen, and can be found in a wide range of colors and patterns to suit any decor style.', response_metadata={'token_usage': {'completion_tokens': 82, 'prompt_tokens': 11, 'total_tokens': 93}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-fb5a5cc1-ded9-4f7b-99c5-30cbeaf32cd5-0', usage_metadata={'input_tokens': 11, 'output_tokens': 82, 'total_tokens': 93})

In [12]:
from langchain.chains import LLMChain, SimpleSequentialChain
from langchain_core.prompts import PromptTemplate

In [13]:
llm = ChatOpenAI(temperature=0.9)

# prompt template 1
prompt = PromptTemplate(
    input_variables=["product"],
    template="What is a good name for a company that makes {product}?",
)
# Chain 1
chain_one = LLMChain(llm=llm, prompt=prompt)

In [14]:
# prompt template 2
second_prompt = PromptTemplate(
    input_variables=["company_name"],
    template="Write a description of a logo for this company: {company_name}, the logo should not contain text at all ",
)
# Chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [15]:
# Create the SimpleSequentialChain
overall_simple_chain = SimpleSequentialChain(
    chains=[chain_one, chain_two], verbose=True
)

In [16]:
overall_simple_chain.invoke(product)

/opt/anaconda3/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new SimpleSequentialChain chain...
Regal Rest Bedding
The logo for Regal Rest Bedding features a stylized, elegant bed frame with intricate detailing and luxurious accents. The design conveys a sense of sophistication and comfort, with a regal crown perched on top of the bed frame symbolizing a feeling of royalty and luxury. The color scheme is a rich combination of deep blues, golds, and purples, adding to the opulent and lavish aesthetic of the logo. Overall, the logo exudes a sense of refined elegance and high-quality bedding products.

> Finished chain.


'The logo for Regal Rest Bedding features a stylized, elegant bed frame with intricate detailing and luxurious accents. The design conveys a sense of sophistication and comfort, with a regal crown perched on top of the bed frame symbolizing a feeling of royalty and luxury. The color scheme is a rich combination of deep blues, golds, and purples, adding to the opulent and lavish aesthetic of the logo. Overall, the logo exudes a sense of refined elegance and high-quality bedding products.'

**Repeat the above twice for different products**

In [21]:
llm = ChatOpenAI(temperature=0.9)

# prompt template 1
prompt = PromptTemplate(
    input_variables=["product"],
    template="Write a poem for this {product}?",
)
# Chain 1
chain_one = LLMChain(llm=llm, prompt=prompt)

# prompt template 2
second_prompt = PromptTemplate(
    input_variables=["company_name"],
    template="Write a song for this company: {company_name}",
)
# Chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

# Create the SimpleSequentialChain
overall_simple_chain = SimpleSequentialChain(
    chains=[chain_one, chain_two], verbose=True
)

overall_simple_chain.invoke(product)



> Entering new SimpleSequentialChain chain...
Oh Queen Size Sheet Set, so regal and grand
In your soft embrace, I rest my weary head
Your silky touch, like a gentle caress
Wrapping me in warmth and comfort, I am truly blessed

Your vibrant colors, a sight to behold
Transforming my room into a palace of gold
With every thread, woven with care
You're a luxurious treat, beyond compare

As I slip between your cool, crisp sheets
I feel a sense of peace and complete
Dreams that dance in my mind's eye
As I drift off to sleep, under your watchful sky

Oh Queen Size Sheet Set, a treasure to behold
You make my bed a sanctuary, a haven to unfold
Thank you for your beauty, your grace, your charm
You're truly a queen, in my humble arms.


In every stitch, in every fold
You bring a touch of elegance, so bold
Oh Queen Size Sheet Set, you're more than just a bed
You're a royal treat, a crown upon my head

With every wash, you stay pristine
A timeless beauty, fit for a queen
I'll cherish you foreverm

{'input': 'Queen Size Sheet Set',
 'output': "\n\nIn every stitch, in every fold\nYou bring a touch of elegance, so bold\nOh Queen Size Sheet Set, you're more than just a bed\nYou're a royal treat, a crown upon my head\n\nWith every wash, you stay pristine\nA timeless beauty, fit for a queen\nI'll cherish you forevermore\nOh Queen Size Sheet Set, my heart you adore\n\nSo I'll lay my head, and rest my soul\nIn the comfort of your regal hold\nOh Queen Size Sheet Set, my loyal friend\nI'll be with you until the very end\n\nOh Queen Size Sheet Set, so regal and grand\nIn your soft embrace, I forever stand\nThank you for being my royal delight\nOh Queen Size Sheet Set, you shine so bright."}

In [22]:
llm = ChatOpenAI(temperature=0.9)

# prompt template 1
prompt = PromptTemplate(
    input_variables=["product"],
    template="What are the main models of the {product}?",
)
# Chain 1
chain_one = LLMChain(llm=llm, prompt=prompt)

# prompt template 2
second_prompt = PromptTemplate(
    input_variables=["company_name"],
    template="Write an advert for this company: {company_name}, highlight its main goals",
)
# Chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

# Create the SimpleSequentialChain
overall_simple_chain = SimpleSequentialChain(
    chains=[chain_one, chain_two], verbose=True
)

overall_simple_chain.invoke(product)



> Entering new SimpleSequentialChain chain...
There are several different models of Queen Size Sheet Set available in the market. Some popular models include:

1. Percale Sheet Set: Made from tightly woven cotton fabric, percale sheets are crisp, cool, and durable. They are a popular choice for those who prefer a classic, hotel-like feel.

2. Sateen Sheet Set: Sateen sheets are made from cotton with a satin weave, giving them a smooth and silky feel. They are known for their luxurious softness and lustrous finish.

3. Microfiber Sheet Set: Microfiber sheets are made from a blend of polyester fibers, making them incredibly soft, lightweight, and wrinkle-resistant. They are a budget-friendly option that is easy to care for.

4. Flannel Sheet Set: Flannel sheets are made from brushed cotton fabric, giving them a cozy and warm feel. They are a popular choice for colder months or those who prefer a softer, more textured sheet.

5. Bamboo Sheet Set: Bamboo sheets are made from bamboo fiber

{'input': 'Queen Size Sheet Set',
 'output': "Looking for the perfect Queen Size Sheet Set for your bed? Look no further! Our company offers a wide range of high-quality sheet sets to suit every preference and budget.\n\nFrom the crisp and durable Percale Sheet Set to the luxurious and silky Sateen Sheet Set, we have something for everyone. If you're looking for a budget-friendly option that is easy to care for, our Microfiber Sheet Set is the perfect choice. Prefer something cozy and warm? Our Flannel Sheet Set will keep you snug during the colder months.\n\nFor those with sensitive skin or allergies, our Bamboo Sheet Set is a sustainable and eco-friendly option that offers unbeatable softness and breathability. No matter your needs, we have the perfect Queen Size Sheet Set for you.\n\nDon't compromise on comfort and quality – choose our company for all your bedding needs. Shop now and experience the difference!"}

## SequentialChain

In [23]:
from langchain.chains import SequentialChain

In [24]:
llm = ChatOpenAI(temperature=0.9)


# Define prompt template 1: Translate a review into English
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review into English:\n\n{review_text}\n\n"
)

# Chain 1: Translate review
chain_one = LLMChain(llm=llm, prompt=first_prompt, output_key="translated_review")


In [25]:
# Define prompt template 2: Summarize a review
second_prompt = ChatPromptTemplate.from_template(
    "Write a summary of the translated review: {translated_review}"
)

# Chain 2: Summarize translated review
chain_two = LLMChain(llm=llm, prompt=second_prompt, output_key="summary")


In [26]:
# prompt template 3: translate to english or other language
third_prompt = ChatPromptTemplate.from_template(
    "Translate the summary into another language: {summary}"
)

# Chain 3: Translate summary to another language
chain_three = LLMChain(llm=llm, prompt=third_prompt, output_key="translated_summary")


In [27]:
# prompt template 4: follow up message that take as inputs the two previous prompts' variables
fourth_prompt = ChatPromptTemplate.from_template(
    "Given the translated review:\n\n{translated_review}\n\n"
    "and its summary:\n\n{summary}\n\n"
    "please generate a follow-up message responding to the review."
)

# Chain 4: Generate follow-up message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt, output_key="follow_up_message")

In [43]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message

overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["review_text"],  # Input variable for the whole process
    output_variables=["translated_review", "summary", "translated_summary", "follow_up_message"],  # Output variables
    verbose=True  # Enable verbose mode for debugging or logging
)

In [40]:
# Example usage: Invoke the overall_chain with a review text
review_text = "This product exceeded my expectations. It is well-built and performs flawlessly."
result = overall_chain.invoke(review_text)



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Human: Translate the following review into Spanish:

This product exceeded my expectations. It is well-built and performs flawlessly.



> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Human: Write a summary of the translated review: Este producto superó mis expectativas. Está bien construido y funciona a la perfección.

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Human: Translate the summary into Spanish: This product exceeded my expectations. It is well-built and works perfectly.

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Human: Given the translated review:

Este producto superó mis expectativas. Está bien construido y funciona a la perfección.

and its summary:

This product exceeded my expectations. It is well-built and works perfectly.

please generate a follow-up message responding

In [41]:
review = df.Review[5]
overall_chain.invoke(review_text)



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Human: Translate the following review into Spanish:

This product exceeded my expectations. It is well-built and performs flawlessly.



> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Human: Write a summary of the translated review: Este producto superó mis expectativas. Está bien construido y funciona perfectamente.

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Human: Translate the summary into Spanish: This product exceeded my expectations. It is well-built and works perfectly.

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Human: Given the translated review:

Este producto superó mis expectativas. Está bien construido y funciona perfectamente.

and its summary:

This product exceeded my expectations. It is well-built and works perfectly.

please generate a follow-up message responding to 

{'review_text': 'This product exceeded my expectations. It is well-built and performs flawlessly.',
 'translated_review': 'Este producto superó mis expectativas. Está bien construido y funciona perfectamente.',
 'summary': 'This product exceeded my expectations. It is well-built and works perfectly.',
 'translated_summary': 'Este producto superó mis expectativas. Está bien construido y funciona perfectamente.',
 'follow_up_message': "Thank you for your positive feedback! We are thrilled to hear that our product exceeded your expectations and that it is working perfectly for you. If you have any questions or need further assistance, please don't hesitate to reach out. We appreciate your support!"}

**Repeat the above twice for different products or reviews**

In [46]:
# Example usage: Invoke the overall_chain with the first review text
review_text1 = "This product exceeded my expectations. It is well-built and performs flawlessly."
print(f"Processing Review 1:\n{review_text1}\n")
result1 = overall_chain.invoke(review_text1)

# Print the detailed result of the first product
print("\nVerbose Result for Product 1:")
print(result1)
print()

# Example usage: Invoke the overall_chain with the second review text
review_text2 = "The customer service was excellent, and the product quality exceeded my expectations."
print(f"Processing Review 2:\n{review_text2}\n")
result2 = overall_chain.invoke(review_text2)

# Print the detailed result of the second product
print("\nVerbose Result for Product 2:")
print(result2)

Processing Review 1:
This product exceeded my expectations. It is well-built and performs flawlessly.



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Human: Translate the following review into Spanish:

This product exceeded my expectations. It is well-built and performs flawlessly.



> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Human: Write a summary of the translated review: Este producto superó mis expectativas. Está bien construido y funciona perfectamente.

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Human: Translate the summary into Spanish: This product exceeded my expectations. It is well-built and works perfectly.

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Human: Given the translated review:

Este producto superó mis expectativas. Está bien construido y funciona perfectamente.

and its summary:

This product exceeded my 

## Router Chain

In [47]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [48]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

In [49]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser

In [50]:
llm = ChatOpenAI(temperature=0)

In [51]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [52]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [53]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [54]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [55]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

In [56]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


"Black body radiation is the electromagnetic radiation emitted by a perfect absorber and emitter of radiation, known as a black body. A black body absorbs all radiation that falls on it and emits radiation at all wavelengths. The spectrum of black body radiation is continuous and depends only on the temperature of the black body. This phenomenon is described by Planck's law, which states that the intensity of radiation emitted by a black body at a given wavelength is proportional to the temperature of the body and inversely proportional to the fourth power of the wavelength."

In [57]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


'The answer to 2 + 2 is 4.'

In [59]:
chain.run("Who was the first man on the moon?")



> Entering new MultiPromptChain chain...
History: {'input': 'Who was the first man on the moon?'}
> Finished chain.


'The first man on the moon was Neil Armstrong, an American astronaut who was part of the Apollo 11 mission in 1969. He famously said, "That\'s one small step for man, one giant leap for mankind," as he became the first human to set foot on the lunar surface.'

**Repeat the above at least once for different inputs and chains executions - Be creative!**